In [ ]:
# common imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# pandas imports
from pandas.plotting import scatter_matrix

from sklearn.model_selection import train_test_split

# display setup
pd.set_option("display.max_columns", None) # the None parameter displays unlimited columns
sns.set(style="whitegrid") # for plots

## 1. Getting the Data

In [ ]:
# read the csv file
df = pd.read_csv("udemy_courses.csv")

In [ ]:
# display the first 5 rows for a quick look
df.head()

In [ ]:
# DataFrame shape (rows, columns)
# understand the amount of data we are working with
df.shape

In [ ]:
# description of data
df.info()

In [ ]:
# check if there are null values
df.isna().sum()

In [ ]:
# summary of the numerical attributes
df.describe()